In [ ]:
import numpy as np
import pandas as pd

SUB_LEN = 100
k = 8
zero_embedding = "NNNNNNNN"

dna2vec = {}
with open('../data/dna2vec/pre-trained DNA-8mers.txt', 'r') as f:
    for line in f:
        k_mer = line[:k]
        line1 = line[k + 1:].split(' ')
        vector = [float(x) for x in line1]
        emb = np.array(vector)
        dna2vec[k_mer] = emb
print(dna2vec)

In [ ]:
file = 'atac_sequence.csv'
df_seq = pd.read_csv(file, index_col=0)
print(df_seq)

In [ ]:
def chunks(l):
    n = SUB_LEN
    y = [l[i:i + n] for i in range(0, len(l), n)]
    return y

def segmentation(seq):
    if len(seq) < k: return np.array(dna2vec[zero_embedding])
    vec = [seq[j:j + k] for j in range(len(seq) - (k - 1))]
    #=====================================================
    for i in range(len(vec)):
        if vec[i] not in dna2vec.keys():
            vec[i] = zero_embedding
    #=====================================================
    vec = [dna2vec[i] for i in vec]
    y = np.array(vec).mean(axis=0)
    return y


num_gene = 0
NUM = 20
def full_map(x):
    global num_gene
    print('In full_map, peak '+str(num_gene)+' ...')
    num_gene +=1
    temp = chunks(x)
    temp = map(segmentation, temp)
    temp = tuple(temp)
    temp = np.stack(temp)
    if temp.shape[0] < NUM:
        sup = np.zeros(shape=(NUM - temp.shape[0], 100), dtype=np.float32)
        temp = np.vstack((temp, sup))
    elif temp.shape[0] > NUM:
        temp = temp[:NUM,:]
    print(temp.shape)
    return temp


def three_quartile_median(arr):
    sorted_arr = np.sort(arr)
    length = len(sorted_arr)
    index = length * 3 / 4
    if index.is_integer():
        return sorted_arr[int(index)]
    else:
        lower_index = int(index)
        upper_index = lower_index + 1
        return (sorted_arr[lower_index] + sorted_arr[upper_index]) / 2

df_seq['sequence'] = df_seq['sequence'].str.upper()
seqs = df_seq['sequence'].values
embs=[]
lengths = []
for seq in seqs:
    emb = full_map(seq).reshape(-1)
    embs.append(emb)
    lengths.append(len(emb))
print(lengths)

In [ ]:
file_out = 'atac_emb.npz'
names = df_seq['interval'].values
np.savez(file_out, peak = names, embedding = np.array(embs))